<a href="https://colab.research.google.com/github/Ezer08/Exploratory-Data-Analysis-and-Visualization-of-Video-Game-Sales-Data/blob/main/Exploratory_Data_Analysis_(EDA)_Video_Games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.express as px




In [2]:
from google.colab import files

# Step 1: Upload the file
uploaded = files.upload()

Saving vgsales.csv to vgsales (2).csv


In [3]:
df= pd.read_csv('vgsales.csv', sep=',')
df.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37



**Understand the Dataset**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [5]:
df.shape

(16598, 11)

In [6]:
df.dtypes

,0
Rank,int64
Name,object
Platform,object
Year,float64
Genre,object
Publisher,object
NA_Sales,float64
EU_Sales,float64
JP_Sales,float64
Other_Sales,float64


In [7]:
#check missing value
df.isnull().sum()

,0
Rank,0
Name,0
Platform,0
Year,271
Genre,0
Publisher,58
NA_Sales,0
EU_Sales,0
JP_Sales,0
Other_Sales,0


In [8]:
#drop missing value
df.dropna(inplace=True)

In [9]:
#Converting the 'Year' Column from Float to Integer Format
df['Year'] = df['Year'].astype(int)


**Exploratory Data Analysis**

In [10]:
# Count the number of games per platform
platform_counts = df['Platform'].value_counts().reset_index()
platform_counts.columns = ['Platform', 'Number of Games']

# Create a horizontal bar plot
fig = px.bar(platform_counts,
             x='Number of Games',
             y='Platform',
             orientation='h',  # Horizontal bars
             title='Number of Games per Platform',
             labels={'Number of Games': 'Number of Games', 'Platform': 'Gaming Platform'},
             color='Number of Games',  # Color bars by the number of games
             color_continuous_scale='Viridis'  # Color scale
            )

# Show the plot
fig.show()

In [11]:
# Create a new column to group the years into bins of 5 years
df['Year_Bin'] = pd.cut(df['Year'], bins=range(int(df['Year'].min()), int(df['Year'].max()) + 5, 5), right=False)

# Convert the Year_Bin intervals to string for better handling by Plotly
df['Year_Bin_Label'] = df['Year_Bin'].apply(lambda x: f"{x.left}-{x.right - 1}")

# Count the number of games in each bin
year_bins_count = df['Year_Bin_Label'].value_counts().reset_index()
year_bins_count.columns = ['Year_Bin', 'Number of Games']

# Sort by the starting year of each bin for chronological order
year_bins_count['Year_Bin'] = year_bins_count['Year_Bin'].apply(lambda x: int(x.split('-')[0]))  # Extract the starting year
year_bins_count = year_bins_count.sort_values(by='Year_Bin')

# Revert the starting year to the original year bin labels
year_bins_count['Year_Bin'] = year_bins_count['Year_Bin'].apply(lambda x: f"{x}-{x+4}")

# Create a vertical bar plot
fig = px.bar(year_bins_count,
             x='Year_Bin',
             y='Number of Games',
             title='Distribution of Games by Year (5-Year Intervals)',
             labels={'Number of Games': 'Number of Games', 'Year_Bin': 'Year Range'},
             color='Number of Games',  # Color bars by the number of games
             color_continuous_scale='Viridis'  # Color scale
            )

# Add labels with the number of games on top of each bar
fig.update_traces(texttemplate='%{y}', textposition='outside')

# Show the plot
fig.show()

In [12]:
import plotly.express as px

# Count the number of games for each genre
genre_count = df['Genre'].value_counts().reset_index()
genre_count.columns = ['Genre', 'Number of Games']

# Create the pie chart with color by Genre
fig = px.pie(genre_count,
             names='Genre',
             values='Number of Games',
             title='Distribution of Games by Genre',
             color='Genre',
             color_discrete_sequence=px.colors.qualitative.Set2)  # Color scheme

# Show the plot
fig.show()


In [13]:
import plotly.express as px
import pandas as pd

# Count the number of games for each publisher
publisher_count = df['Publisher'].value_counts().reset_index()
publisher_count.columns = ['Publisher', 'Number of Games']

# Get the top 9 publishers and group the rest as 'Others'
top_publishers = publisher_count.head(9)
others = publisher_count.iloc[9:].sum(numeric_only=True)

# Append 'Others' to the data
others_data = pd.DataFrame({'Publisher': ['Others'], 'Number of Games': [others['Number of Games']]})
final_publisher_data = pd.concat([top_publishers, others_data])

# Create the pie chart
fig = px.pie(final_publisher_data,
             names='Publisher',
             values='Number of Games',
             title='Top 9 Publishers and Others',
             color='Publisher',
             color_discrete_sequence=px.colors.qualitative.Set3)

# Show the plot
fig.show()


In [14]:
import plotly.express as px
import pandas as pd

# Sum the sales for each region (NA_Sales, EU_Sales, JP_Sales)
region_sales = df[['NA_Sales', 'EU_Sales', 'JP_Sales']].sum()

# Create a DataFrame for the pie chart
region_sales_df = pd.DataFrame(region_sales).reset_index()
region_sales_df.columns = ['Region', 'Sales']

# Create the pie chart with interactive hover data
# Changed hover_data from dictionary to list
fig = px.pie(region_sales_df,
             names='Region',
             values='Sales',
             title='Sales Distribution by Region',
             color='Region',
             color_discrete_sequence=['red', 'yellow', 'green'],
             hover_data=['Region', 'Sales'])  # Show sales data on hover


# Show the plot
fig.show()

**Key insights**

**Sales Distribution Analysis**

*Top 10 selling game globally*

In [15]:
import plotly.express as px

# Sort the DataFrame by Global Sales and select the Top 10
df_top10 = df.sort_values(by="Global_Sales", ascending=False).head(10)

# Highlight the top-selling game (1st in the top 10)
top_game = df_top10.iloc[0]["Name"]
df_top10['Highlight'] = df_top10['Name'].apply(lambda x: 'Top-Selling' if x == top_game else 'Other Games')

# Create an interactive bar chart
fig = px.bar(
    df_top10,
    x="Name",
    y="Global_Sales",
    color="Highlight",
    title="Top 10 Best-Selling Games Globally",
    labels={"Global_Sales": "Global Sales (in Millions)", "Name": "Game"},
    text="Global_Sales",
    color_discrete_map={"Top-Selling": "gold", "Other Games": "lightblue"}
)

# Format the text and layout
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(
    xaxis_title="Game",
    yaxis_title="Global Sales (in Millions)",
    xaxis={'categoryorder': 'total descending'},
    showlegend=False
)

# Show the interactive plot
fig.show()


*Top 10 Best-Selling Games by region*

In [16]:
import plotly.express as px

# Define a function to generate the top 10 interactive bar chart for any region
def plot_top10_sales(region_column, region_name):
    # Sort by the selected regional sales column and take the top 10
    df_top10_region = df.sort_values(by=region_column, ascending=False).head(10)

    # Highlight the top-selling game in the selected region
    top_game_region = df_top10_region.iloc[0]["Name"]
    df_top10_region['Highlight'] = df_top10_region['Name'].apply(
        lambda x: 'Top-Selling' if x == top_game_region else 'Other Games'
    )

    # Create an interactive bar chart
    fig = px.bar(
        df_top10_region,
        x="Name",
        y=region_column,
        color="Highlight",
        title=f"Top 10 Best-Selling Games in {region_name}",
        labels={region_column: f"{region_name} Sales (in Millions)", "Name": "Game"},
        text=region_column,
        color_discrete_map={"Top-Selling": "gold", "Other Games": "lightblue"}
    )

    # Format the layout
    fig.update_traces(texttemplate='%{text}', textposition='outside')
    fig.update_layout(
        xaxis_title="Game",
        yaxis_title=f"{region_name} Sales (in Millions)",
        xaxis={'categoryorder': 'total descending'},
        showlegend=False
    )

    fig.show()

# Plot for NA, EU, JP, and Other Sales
plot_top10_sales("NA_Sales", "North America")
plot_top10_sales("EU_Sales", "Europe")
plot_top10_sales("JP_Sales", "Japan")
plot_top10_sales("Other_Sales", "Other Regions")


*Top 10 Best-Selling Games: Sales Comparison by Region*

In [17]:
import pandas as pd
import plotly.express as px

# Melt the DataFrame to combine regional sales into one column
df_melted = df.sort_values(by="Global_Sales", ascending=False).head(10).melt(
    id_vars=["Name"],
    value_vars=["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"],
    var_name="Region",
    value_name="Sales"
)

# Map region names for better readability
region_mapping = {
    "NA_Sales": "North America",
    "EU_Sales": "Europe",
    "JP_Sales": "Japan",
    "Other_Sales": "Other Regions"
}
df_melted["Region"] = df_melted["Region"].map(region_mapping)

# Create an interactive grouped bar chart
fig = px.bar(
    df_melted,
    x="Name",
    y="Sales",
    color="Region",
    title="Top 10 Best-Selling Games: Sales Comparison by Region",
    labels={"Sales": "Sales (in Millions)", "Name": "Game"},
    barmode="group",
    text="Sales",
    color_discrete_map={
        "North America": "red",
        "Europe": "blue",
        "Japan": "green",
        "Other Regions": "orange"
    }
)

# Format the layout
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    xaxis_title="Game",
    yaxis_title="Sales (in Millions)",
    xaxis={'categoryorder': 'total descending'}
)

# Show the chart
fig.show()


**Regional Analysis**

In [18]:

# Melt the DataFrame to combine regional sales into one column
df_melted = df.melt(
    id_vars=["Name", "Global_Sales"],
    value_vars=["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"],
    var_name="Region",
    value_name="Sales"
)

# Map region names for better readability
region_mapping = {
    "NA_Sales": "North America",
    "EU_Sales": "Europe",
    "JP_Sales": "Japan",
    "Other_Sales": "Other Regions"
}
df_melted["Region"] = df_melted["Region"].map(region_mapping)

In [19]:
# Plot 1: Contribution of Each Region to Global Sales
region_contribution = df_melted.groupby("Region")["Sales"].sum().reset_index()

fig1 = px.pie(
    region_contribution,
    names="Region",
    values="Sales",
    title="Regional Contribution to Global Sales",
    color="Region",
    color_discrete_map={
        "North America": "red",
        "Europe": "blue",
        "Japan": "green",
        "Other Regions": "orange"
    }
)
fig1.show()

**Platform Performance**

*Total Sales by Platform*

In [22]:
# Group the data by platform and calculate total sales for each platform globally
platform_sales = df.groupby("Platform")["Global_Sales"].sum().reset_index()

# Sort the platform sales by global sales
platform_sales_sorted = platform_sales.sort_values(by="Global_Sales", ascending=False)

# Plot: Rank Platforms by Global Sales with color based on sales concentration
fig2 = px.bar(
    platform_sales_sorted,
    x="Platform",
    y="Global_Sales",
    title="Ranking Platforms by Global Sales",
    labels={"Global_Sales": "Global Sales (in Millions)", "Platform": "Platform"},
    color="Global_Sales",  # Color based on Global_Sales
    color_continuous_scale=px.colors.diverging.RdYlGn,  # Red to Green color scale
    range_color=[platform_sales_sorted["Global_Sales"].min(), platform_sales_sorted["Global_Sales"].max()]  # Adjust color range
)
fig2.update_layout(xaxis_title="Platform", yaxis_title="Global Sales (in Millions)")
fig2.show()

In [23]:
# Group the data by platform and calculate total sales by region
platform_sales = df.groupby("Platform")[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]].sum().reset_index()

# Melt the DataFrame to make it suitable for a stacked bar chart
df_platform_melted = platform_sales.melt(
    id_vars=["Platform"],
    value_vars=["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"],
    var_name="Region",
    value_name="Sales"
)

# Map region names for readability
region_mapping = {
    "NA_Sales": "North America",
    "EU_Sales": "Europe",
    "JP_Sales": "Japan",
    "Other_Sales": "Other Regions"
}
df_platform_melted["Region"] = df_platform_melted["Region"].map(region_mapping)

# Create the stacked bar chart
fig = px.bar(
    df_platform_melted,
    x="Platform",
    y="Sales",
    color="Region",
    title="Total Sales by Platform (Global) with Regional Breakdown",
    labels={"Sales": "Sales (in Millions)", "Platform": "Platform"},
    color_discrete_sequence=px.colors.qualitative.Set2,
    barmode="stack"
)

# Show the plot
fig.update_layout(xaxis_title="Platform", yaxis_title="Sales (in Millions)")
fig.show()

**Genre performance**

In [24]:
# Group the data by genre and calculate total sales by region and globally
genre_sales = df.groupby("Genre")[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"]].sum().reset_index()

# ------------------------------
# 1. Most Successful Genre Globally
fig1 = px.bar(
    genre_sales,
    x="Genre",
    y="Global_Sales",
    title="Most Successful Genre Globally",
    labels={"Global_Sales": "Global Sales (in Millions)", "Genre": "Genre"},
    color="Genre",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig1.update_layout(xaxis_title="Genre", yaxis_title="Global Sales (in Millions)")
fig1.show()

In [25]:
# Group the data by genre and calculate total sales by region and globally
genre_sales = df.groupby("Genre")[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]].sum().reset_index()

# Melt the data to create a stacked bar chart
df_genre_melted = genre_sales.melt(
    id_vars=["Genre"],
    value_vars=["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"],
    var_name="Region",
    value_name="Sales"
)

# Map region names for readability
df_genre_melted["Region"] = df_genre_melted["Region"].map({
    "NA_Sales": "North America",
    "EU_Sales": "Europe",
    "JP_Sales": "Japan",
    "Other_Sales": "Other Regions"
})

# Plot: Genre Sales by Region (Stacked Bar)
fig = px.bar(
    df_genre_melted,
    x="Genre",
    y="Sales",
    color="Region",
    title="Genre Sales by Region (Stacked Bar)",
    labels={"Sales": "Sales (in Millions)", "Genre": "Genre"},
    color_discrete_sequence=px.colors.qualitative.Set2,
    barmode="stack"
)

# Show the plot
fig.update_layout(xaxis_title="Genre", yaxis_title="Sales (in Millions)")
fig.show()

**Publisher Analysis**

In [26]:
# Group the data by publisher and calculate total sales by region and globally
publisher_sales = df.groupby("Publisher")[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"]].sum().reset_index()

# ------------------------------
# 1. Top Publishers by Global Sales
fig1 = px.bar(
    publisher_sales.sort_values("Global_Sales", ascending=False).head(10),
    x="Publisher",
    y="Global_Sales",
    title="Top Publishers by Global Sales",
    labels={"Global_Sales": "Global Sales (in Millions)", "Publisher": "Publisher"},
    color="Publisher",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig1.update_layout(xaxis_title="Publisher", yaxis_title="Global Sales (in Millions)")
fig1.show()


In [27]:
import pandas as pd
import plotly.express as px

# Group the data by publisher and calculate total sales by region and globally
publisher_sales = df.groupby("Publisher")[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"]].sum().reset_index()

# Get top 10 publishers by global sales
top_10_publishers = publisher_sales.sort_values("Global_Sales", ascending=False).head(10)

# Melt the data for regional sales comparison
df_publisher_melted = top_10_publishers.melt(
    id_vars=["Publisher"],
    value_vars=["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"],
    var_name="Region",
    value_name="Sales"
)

# Map region names for readability
df_publisher_melted["Region"] = df_publisher_melted["Region"].map({
    "NA_Sales": "North America",
    "EU_Sales": "Europe",
    "JP_Sales": "Japan",
    "Other_Sales": "Other Regions"
})

# Plot: Publisher Sales by Region (Grouped Bar)
fig = px.bar(
    df_publisher_melted,
    x="Publisher",
    y="Sales",
    color="Region",
    title="Top 10 Publishers by Regional Sales",
    labels={"Sales": "Sales (in Millions)", "Publisher": "Publisher"},
    barmode="group",  # Group bars for each publisher
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Show the plot
fig.update_layout(xaxis_title="Publisher", yaxis_title="Sales (in Millions)")
fig.show()


In [28]:
import pandas as pd
import plotly.express as px

# ------------------------------
# 1. Top-selling Games by Year (Global Sales)
# Find the top-selling game for each year
top_game_by_year = df.loc[df.groupby("Year")["Global_Sales"].idxmax()][["Year", "Name", "Global_Sales"]]

# Plot: Top-selling Games by Year
fig1 = px.bar(
    top_game_by_year,
    x="Year",
    y="Global_Sales",
    color="Name",
    title="Top-selling Games by Year (Global Sales)",
    labels={"Global_Sales": "Global Sales (in Millions)", "Year": "Year", "Name": "Game"},
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig1.update_layout(xaxis_title="Year", yaxis_title="Global Sales (in Millions)")
fig1.show()

# ------------------------------

In [29]:

# 2. Global Sales Trends by Year
# Calculate total sales per year
yearly_sales = df.groupby("Year")[["Global_Sales"]].sum().reset_index()

# Plot: Global Sales Trends by Year
fig2 = px.line(
    yearly_sales,
    x="Year",
    y="Global_Sales",
    title="Global Sales Trends by Year",
    labels={"Global_Sales": "Global Sales (in Millions)", "Year": "Year"}
)
fig2.update_layout(xaxis_title="Year", yaxis_title="Global Sales (in Millions)")
fig2.show()

# ------------------------------

In [30]:

# 3. Top Publishers by Year (Global Sales)
# Find the top publisher for each year by total sales
top_publisher_by_year = df.groupby(["Year", "Publisher"])["Global_Sales"].sum().reset_index()
top_publisher_by_year = top_publisher_by_year.loc[top_publisher_by_year.groupby("Year")["Global_Sales"].idxmax()]

# Plot: Top Publishers by Year (Global Sales)
fig3 = px.bar(
    top_publisher_by_year,
    x="Year",
    y="Global_Sales",
    color="Publisher",
    title="Top Publishers by Year (Global Sales)",
    labels={"Global_Sales": "Global Sales (in Millions)", "Year": "Year", "Publisher": "Publisher"},
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig3.update_layout(xaxis_title="Year", yaxis_title="Global Sales (in Millions)")
fig3.show()

# ------------------------------